# Calculations implantation depths

In [1]:
%run GenerateStoppingPower.ipynb

SRIM stopping power successfully calculated
	Calculate energy loss as a function of traversed target material with function 'GetElossAfterX(E_init, x)'
	Calculate traversed target material as a function of energy loss 'GetXAfterEloss(E_init, E_after)'
	Calculate range in material as a function of energy 'GetRange(E_init)'
Successfully imported 'GenerateStoppingPower'
	Class to Generate SRIM Stopping Power and calculate energy loss and traversed material.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.linalg
from matplotlib import colors
import matplotlib as mpl
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.backends.backend_pdf import PdfPages
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd

color_list = plt.rcParams['axes.prop_cycle'].by_key()['color'];
#Customising style
#mpl.rc('lines', linewidth=2, color='r')
mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['font.size'] = 24
#mpl.rcParams['savefig.format'] = "eps"
#mpl.rcParams['savefig.dpi'] = 100
#mpl.rcParams['text.usetex'] = True
#mpl.rcParams['figure.figsize'] = (16,9)

## Fast-Alphas

The kinematics as calculated in [notebook](TASCA_U225.ipynb) is assumed.

Ion: U-225

In [3]:
E_kin = 45 #MeV

In [4]:
# Construct a layer of SiO2, arbitrary width (as it is not relevant for these calcs)
layer = Layer({
    'Si': {
        'stoich': 1,
        #'E_d': 35.0, # Displacement Energy
        #'lattice': 0.0,
        #'surface': 3.0
    },
    'O': {
        'stoich': 2,
        #'E_d': 20.0, # Displacement Energy
        #'lattice': 0.0,
        #'surface': 3.0
    }
}, density=2.32, width=10000.0)

In [5]:
# Construct ion
A, E_per_A_max = 225, 7e6 # nucleon number, eV 
energy_max = A*E_per_A_max
#print("Max energy=", energy_max)
ion = Ion('U', energy=energy_max, mass=A)

U225 = GenerateStoppingPower(layer, ion)

SRIM stopping power successfully calculated
	Calculate energy loss as a function of traversed target material with function 'GetElossAfterX(E_init, x)'
	Calculate traversed target material as a function of energy loss 'GetXAfterEloss(E_init, E_after)'
	Calculate range in material as a function of energy 'GetRange(E_init)'


In [12]:
print("Range =", U225.GetRange(45, from_sr=True), "µm")

Range = 7.429001481209475 µm


## Fl-289

In [7]:
# Construct ion
A, E_per_A_max = 289, 7e6 # nucleon number, eV 
energy_max = A*E_per_A_max
#print("Max energy=", energy_max)
ion = Ion('U', energy=energy_max, mass=A)

Fl289 = GenerateStoppingPower(layer, ion, z_ion=114)

Extrapolating stopping power for ion with z= 114
Mean scaling factor: 1.3309325475011262
SRIM stopping power successfully calculated
	Calculate energy loss as a function of traversed target material with function 'GetElossAfterX(E_init, x)'
	Calculate traversed target material as a function of energy loss 'GetXAfterEloss(E_init, E_after)'
	Calculate range in material as a function of energy 'GetRange(E_init)'


In [13]:
print("Range =", Fl289.GetRange(22, from_sr=True), "µm")

Range = 3.096210212861479 µm
